## Import tensorflow

In [1]:
import tensorflow as tf
import numpy as np

## Specifying which version of library is used

In [2]:
print(tf.__version__)

2.3.1


### Make tensorflow compitable with olderversion

In [3]:
tf.compat.v1.disable_eager_execution()

## Load mnist dataset

In [4]:
mnist = tf.keras.datasets.mnist

## Split the dataset into testing and training

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_val = x_test[7000:]
y_val = y_test[7000:]

x_test = x_test[:7000]
y_test = y_test[:7000]

### Defining some variables

In [6]:
batch_size = 100
l_mnist = 784
l_labels = 10
learning_rate = 0.5

### Define batch generator function

In [7]:
def batch_generator(X, Y, batch_size):
    indices = np.arange(len(X))
    batch = []
    while True:
        np.random.shuffle(indices)
        for i in indices:
            batch.append(i)
            if len(batch)==batch_size:
                yield X[batch], Y[batch]
                batch =[]

### Build models using 0 layer

In [8]:
img = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_mnist])
ans = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_labels])

# Weight and baise
weight = tf.Variable(tf.random.normal([l_mnist,l_labels], stddev=0.1))
baise = tf.Variable(tf.random.normal([l_labels], stddev=0.1))

# calculating probability
probs = tf.nn.softmax(tf.matmul(img, weight) + baise)

# calculating entrophy
xEntrophy = tf.reduce_mean(-tf.reduce_sum(ans*tf.math.log(probs)))

# build train model using GradientDescentOptimizer
train = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(xEntrophy)

# Finding Accuracy
numCorrect = tf.equal(tf.argmax(probs,1), tf.argmax(ans,1))
accuracy = tf.reduce_mean(tf.cast(numCorrect, tf.float32))

# Create Session for tensorflow
session = tf.compat.v1.Session()
session.run(tf.compat.v1.global_variables_initializer())

# training start
train_generator = batch_generator(x_train, y_train, batch_size)

weigth = [1000, 2000]
for w in weigth:
    num_of_weigth_change = w
    for i in range(num_of_weigth_change):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size), ys] = 1
        session.run(train, feed_dict={img:xsn, ans:ysn})

    sumAcc = 0
    for i in range(1000):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size),ys] = 1
        sumAcc += session.run(accuracy, {img:xsn, ans:ysn})
    print("Accuracy for for number of weigth {}: {}".format(num_of_weigth_change, sumAcc))

Accuracy for for number of weigth 1000: 99.16999986208975
Accuracy for for number of weigth 2000: 98.0099998600781


### Build models using 1 layer using relu function

In [9]:
img = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_mnist])
ans = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_labels])

nFirstLayer = 20
# U, bU are weigth and baise for vector leading input to  first hidden layer
U = tf.Variable(tf.random.normal([l_mnist, nFirstLayer], stddev=0.1))
bU = tf.Variable(tf.random.normal([nFirstLayer], stddev=0.1))

# V, bV are weigth and baise for vector leading hidden layer to output layer
V = tf.Variable(tf.random.normal([nFirstLayer, l_labels], stddev=0.1))
bV = tf.Variable(tf.random.normal([l_labels], stddev=0.1))

# Affine tranformation for first layer and passed through relu function
L1Output = tf.matmul(img, U) + bU
L1Output = tf.nn.relu(L1Output)

# Calculating probability by using softmax function
probs = tf.nn.softmax(tf.matmul(L1Output, V) + bV)
xEntrophy = tf.reduce_mean(-tf.reduce_sum(ans*tf.math.log(probs)))

# build train model using GradientDescentOptimizer
train = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(xEntrophy)

# Finding Accuracy
numCorrect = tf.equal(tf.argmax(probs,1), tf.argmax(ans,1))
accuracy = tf.reduce_mean(tf.cast(numCorrect, tf.float32))

# Create Session for tensorflow
session = tf.compat.v1.Session()
session.run(tf.compat.v1.global_variables_initializer())


# training start
train_generator = batch_generator(x_train, y_train, batch_size)

weigth = [1000, 2000]
for w in weigth:
    epos = w
    for i in range(epos):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size), ys] = 1
        session.run(train, feed_dict={img:xsn, ans:ysn})

    sumAcc = 0
    for i in range(1000):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size),ys] = 1
        sumAcc += session.run(accuracy, {img:xsn, ans:ysn})
    print("Accuracy for for number of weigth {}: {}".format(epos, sumAcc))

Accuracy for for number of weigth 1000: 98.16000000201166
Accuracy for for number of weigth 2000: 98.77999989688396


### Build models using 1 layer using signmod function

In [10]:
img = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_mnist])
ans = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_labels])

nFirstLayer = 20
# U, bU are weigth and baise for vector leading input to  first hidden layer
U = tf.Variable(tf.random.normal([l_mnist, nFirstLayer], stddev=0.1))
bU = tf.Variable(tf.random.normal([nFirstLayer], stddev=0.1))

# V, bV are weigth and baise for vector leading hidden layer to output layer
V = tf.Variable(tf.random.normal([nFirstLayer, l_labels], stddev=0.1))
bV = tf.Variable(tf.random.normal([l_labels], stddev=0.1))

# Affine tranformation for first layer and passed through relu function
L1Output = tf.matmul(img, U) + bU
L1Output = tf.nn.sigmoid(L1Output)

# Calculating probability by using softmax function
probs = tf.nn.softmax(tf.matmul(L1Output, V) + bV)
xEntrophy = tf.reduce_mean(-tf.reduce_sum(ans*tf.math.log(probs)))

# build train model using GradientDescentOptimizer
train = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(xEntrophy)

# Finding Accuracy
numCorrect = tf.equal(tf.argmax(probs,1), tf.argmax(ans,1))
accuracy = tf.reduce_mean(tf.cast(numCorrect, tf.float32))

# Create Session for tensorflow
session = tf.compat.v1.Session()
session.run(tf.compat.v1.global_variables_initializer())


# training start
train_generator = batch_generator(x_train, y_train, batch_size)

weigth = [1000, 2000]
for w in weigth:
    epos = w
    for i in range(epos):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size), ys] = 1
        session.run(train, feed_dict={img:xsn, ans:ysn})

    sumAcc = 0
    for i in range(1000):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size),ys] = 1
        sumAcc += session.run(accuracy, {img:xsn, ans:ysn})
    print("Accuracy for for number of weigth {}: {}".format(epos, sumAcc))

Accuracy for for number of weigth 1000: 98.61999990046024
Accuracy for for number of weigth 2000: 99.20999990403652


### Build Model using 2 layer using Relu function

In [12]:
img = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_mnist])
ans = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_labels])

nFirstLayer = 20
# U, bU are weigth and baise for vector leading input to  first hidden layer
U = tf.Variable(tf.random.normal([l_mnist, nFirstLayer], stddev=0.1))
bU = tf.Variable(tf.random.normal([nFirstLayer], stddev=0.1))

# U2, bU2 are weigth and baise for vector leading first hidden layer to second hidden layer
U2 = tf.Variable(tf.random.normal([l_mnist, nFirstLayer], stddev=0.1))
bU2 = tf.Variable(tf.random.normal([nFirstLayer], stddev=0.1))

# V, bV are weigth and baise for vector leading hidden layer to output layer
V = tf.Variable(tf.random.normal([nFirstLayer, l_labels], stddev=0.1))
bV = tf.Variable(tf.random.normal([l_labels], stddev=0.1))

# Affine tranformation for first layer and passed through relu function
L1Output = tf.matmul(img, U) + bU
L1Output = tf.nn.relu(L1Output)

# Affine tranformation for second layer and passed through relu function
L2Output = tf.matmul(img, U2) + bU2
L2Output = tf.nn.relu(L2Output)

# Calculating probability by using softmax function
probs = tf.nn.softmax(tf.matmul(L2Output, V) + bV)

# Calculating Entrophy 
xEntrophy = tf.reduce_mean(-tf.reduce_sum(ans*tf.math.log(probs)))

# build train model using GradientDescentOptimizer
train = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(xEntrophy)

# Finding Accuracy
numCorrect = tf.equal(tf.argmax(probs,1), tf.argmax(ans,1))
accuracy = tf.reduce_mean(tf.cast(numCorrect, tf.float32))

# Create Session for tensorflow
session = tf.compat.v1.Session()
session.run(tf.compat.v1.global_variables_initializer())

# start training
train_generator = batch_generator(x_train, y_train, batch_size)

weigth = [1000, 2000]
for w in weigth:
    epos = w
    for i in range(epos):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size), ys] = 1
        session.run(train, feed_dict={img:xsn, ans:ysn})

    sumAcc = 0
    for i in range(1000):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size),ys] = 1
        sumAcc += session.run(accuracy, {img:xsn, ans:ysn})
    print("Accuracy for for number of weigth {}: {}".format(epos, sumAcc))

Accuracy for for number of weigth 1000: 98.59999986179173
Accuracy for for number of weigth 2000: 99.31999995745718


### Build Model using 2 layer using Sigmod function

In [13]:
img = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_mnist])
ans = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, l_labels])

nFirstLayer = 20
# U, bU are weigth and baise for vector leading input to  first hidden layer
U = tf.Variable(tf.random.normal([l_mnist, nFirstLayer], stddev=0.1))
bU = tf.Variable(tf.random.normal([nFirstLayer], stddev=0.1))

# U2, bU2 are weigth and baise for vector leading first hidden layer to second hidden layer
U2 = tf.Variable(tf.random.normal([l_mnist, nFirstLayer], stddev=0.1))
bU2 = tf.Variable(tf.random.normal([nFirstLayer], stddev=0.1))

# V, bV are weigth and baise for vector leading hidden layer to output layer
V = tf.Variable(tf.random.normal([nFirstLayer, l_labels], stddev=0.1))
bV = tf.Variable(tf.random.normal([l_labels], stddev=0.1))

# Affine tranformation for first layer and passed through relu function
L1Output = tf.matmul(img, U) + bU
L1Output = tf.nn.sigmoid(L1Output)

# Affine tranformation for second layer and passed through relu function
L2Output = tf.matmul(img, U2) + bU2
L2Output = tf.nn.sigmoid(L2Output)

# Calculating probability by using softmax function
probs = tf.nn.softmax(tf.matmul(L2Output, V) + bV)

# Calculating Entrophy 
xEntrophy = tf.reduce_mean(-tf.reduce_sum(ans*tf.math.log(probs)))

# build train model using GradientDescentOptimizer
train = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(xEntrophy)

# Finding Accuracy
numCorrect = tf.equal(tf.argmax(probs,1), tf.argmax(ans,1))
accuracy = tf.reduce_mean(tf.cast(numCorrect, tf.float32))

# Create Session for tensorflow
session = tf.compat.v1.Session()
session.run(tf.compat.v1.global_variables_initializer())

# Start training
train_generator = batch_generator(x_train, y_train, batch_size)

weigth = [1000, 2000]
for w in weigth:
    epos = w
    for i in range(epos):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size), ys] = 1
        session.run(train, feed_dict={img:xsn, ans:ysn})

    sumAcc = 0
    for i in range(1000):
        xs, ys = next(train_generator)
        xsn = xs.reshape(batch_size,l_mnist)
        ysn = np.zeros((ys.size, ys.max()+1))
        ysn[np.arange(ys.size),ys] = 1
        sumAcc += session.run(accuracy, {img:xsn, ans:ysn})
    print("Accuracy for for number of weigth {}: {}".format(epos, sumAcc))

Accuracy for for number of weigth 1000: 99.51999997720122
Accuracy for for number of weigth 2000: 98.65999989770353


# Using Keras Liberary

### Build model using 1 layer relu activation function

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, batch_size=100, validation_data=(x_val, y_val), verbose=0)
test_loss, test_acc = model.evaluate(x_train,  y_train, verbose=2)
print('\nTest accuracy:', test_acc)
history.history


Test accuracy: 0.9948


{'loss': [0.28590078176930545,
  0.11627211503994962,
  0.08070516010591139,
  0.05817798526026308,
  0.04602849234283591,
  0.037039140319490495,
  0.028143381598832395,
  0.0241859797766665,
  0.020747485711229578,
  0.01571817357544205],
 'accuracy': [0.9179,
  0.96533334,
  0.97515,
  0.9820833,
  0.98546666,
  0.98825,
  0.9909,
  0.99225,
  0.99325,
  0.99486667],
 'val_loss': [0.09726542321344217,
  0.07459922333558401,
  0.05586053871860107,
  0.051744054754575095,
  0.049084736655155815,
  0.0559089469568183,
  0.04732186727924272,
  0.053257337453154224,
  0.043421285166793194,
  0.05249617339771551],
 'val_accuracy': [0.96933335,
  0.97933334,
  0.98333335,
  0.98733336,
  0.9856667,
  0.9813333,
  0.989,
  0.9853333,
  0.98933333,
  0.988]}

### Build model using 1 layer sigmoid activation function

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='sigmoid'),
    tf.keras.layers.Dense(128, activation='sigmoid'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, batch_size=100, validation_data=(x_val, y_val), verbose=0)
test_loss, test_acc = model.evaluate(x_train,  y_train, verbose=2)
print('\nTest accuracy:', test_acc)
history.history


Test accuracy: 0.99023336


{'loss': [0.6279800433417161,
  0.2314765346298615,
  0.17229543319592874,
  0.13659964537558456,
  0.11114081292413175,
  0.09136032730651399,
  0.07554899915431937,
  0.06414874405134469,
  0.053563755290039504,
  0.0456554550724104],
 'accuracy': [0.8425,
  0.93275,
  0.94906664,
  0.9600833,
  0.96753335,
  0.9730167,
  0.97781664,
  0.98116666,
  0.9841833,
  0.9867667],
 'val_loss': [0.19015120243032774,
  0.13036363509794077,
  0.10886791038016479,
  0.0919844876974821,
  0.0726894499734044,
  0.06757935583591461,
  0.06202606704706947,
  0.05732048371185859,
  0.05276733363668124,
  0.04905499958743652],
 'val_accuracy': [0.94666666,
  0.96166664,
  0.971,
  0.97466666,
  0.9773333,
  0.98066664,
  0.981,
  0.982,
  0.98433334,
  0.986]}